In [1]:
# !pip -q install tensorflow==2.8 > log
# !apt -q install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 >> log
!pip -q install -U tfds-nightly tensorflow_addons tensorflow >> log

Program to optimize paramaters eg period for price bitcoin prediction.

In [2]:
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os
import tensorflow_addons as tfa, tensorflow as tf, tensorflow.keras as keras, tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, MaxPooling2D, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.layers import SimpleRNN, Flatten, Dense, RNN, LSTM, TimeDistributed
from keras import backend as K
from keras.layers import Layer

sample=pd.read_csv('tXY.csv', index_col='id');
train = sample[0:200000];
test= sample[200000:230000];

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
tXY1 = train.copy()
tXY = train.pct_change().fillna(0)

In [4]:
def getData(seq_size):
    K, (N, p), Nx, Ny = seq_size, tXY.shape, 20000, len(test)   # samples, dataset dim, train set size, forecast set size
    LtX, LtY = [], []
    for i in range(N-Ny-K, N-Ny):                     # populate K samples with past X series and future Y series
      LtX.append(tXY.iloc[(i-Nx):i, :].values)        # X: historical 7Dim observations for Nx steps behind
      LtY.append(tXY.loc[i:(i+Ny-1),'Close'].values)  # Y: future closing prices for Ny steps ahead
    taX, taY = np.array(LtX), np.array(LtY)           # training arrays past input X and future output Y
    #print(f'taX.shape=(K,Nx,p)={taX.shape}; taY=(K,Ny)={taY.shape}')  # convert to 3-tensors
    return taX, taY,p,Ny,Nx


In [5]:
pip install pyswarms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.0 MB/s eta 0:00:00


In [6]:
import torch
import pyswarms as ps
import numpy as np
from pyswarms.utils.plotters import plot_cost_history
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Conv1D, GRU, Input, BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.models import Model
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error,r2_score,mean_absolute_error
# Define the fitness function to be optimized

# Define the fitness function to be optimized
def fitness_function(seq_size):

  taX, taY,p, Ny,Nx=getData(seq_size)

  tf.random.set_seed(0)   # always seed your experiments
  Init = keras.initializers.GlorotUniform(seed=0) # seed all that you can

  input_shape = (None, p)

# Create the input layer
  input_layer = Input(shape=input_shape, name='input_layer')
  tf.random.set_seed(0)   # always seed your experiments
  tf.random.set_seed(0)   # always seed your experiments
  Init = keras.initializers.GlorotUniform(seed=0) # seed all that you can

  input_shape = (None, p)

# Create the input layer
  input_layer = Input(shape=input_shape, name='input_layer')

# Add Conv1D layer, kernel_size of 5 proved to do well, more or less tended to decrease performance
  conv_layer = Conv1D(filters=200, kernel_size=5, activation='selu', name='conv_layer', kernel_initializer=Init)(input_layer)

# Feed Conv1D results into recurrent layers
  lstm_layer1 = LSTM(units=200, return_sequences=True, kernel_initializer=Init, recurrent_initializer=Init, name='LSTM1')(conv_layer)
  dropout1 = Dropout(0.2, name='dropout1', seed=42)(lstm_layer1)

  lstm_layer2 = LSTM(units=200, return_sequences=False, kernel_initializer=Init, recurrent_initializer=Init, name='LSTM2')(dropout1)
  dropout2 = Dropout(0.2, name='dropout2', seed=42)(lstm_layer2)

# Add the output layer, tanh to keep the output bounded to represent percentage change
  output_layer = Dense(activation='tanh', units=Ny, kernel_initializer=Init, name='output')(dropout2)

# Create the model
  model = Model(inputs=input_layer, outputs=output_layer, name='RNN_model')

  model.summary()

# Course adjustment
  lrs = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=.1, decay_steps=10000, decay_rate=0.01)
  opt = keras.optimizers.Adam(learning_rate=lrs, amsgrad=True)

# Mean absolute error to be robust against major swings
  model.compile(loss="mean_absolute_error", optimizer=opt, metrics=['cosine_similarity'])
  history = model.fit(taX, taY, epochs=4, batch_size=16)


  tX_recent = tXY  # most recent history of the coin as percentage changes
  tX_recent2 = tXY1.iloc[-Nx:,:]  # most recent history of the coin in original form
  print(f'tX_recent.shape=(Nx,p)={tX_recent.shape}')

  fY = model.predict(tX_recent.values[np.newaxis,...]).flatten()  # the model expects a 3-tensor (K=1,Nx,p)
# Restore percentages and clip to previously observed ranges
  fY2 = pd.DataFrame(fY+1).clip(lower=0.99, upper=1.01)
# Replace first array with last known close price to restore actual price
  fY2.iloc[0] *= tX_recent2.iloc[-1].Close
# Restore price fully with a cumulative product
  test['ClosePredict2'] = fY2.cumprod().to_numpy().flatten()
  mae = mean_absolute_error(test['Close'] , test['ClosePredict2'] )
  return mae



def avg(_x):
    return (_x[0]+_x[1])/2.

def costf_pso(_X):  # PSO minimizes
    """
    _X is the list of particles called by the pyswarms library
    (S, M) = _X.shape
    S number of particles
    M number of dimensions, i.e. number of independent variables, hyperparameters
    """


    S = _X.shape[0]  # number of particles
    # each particle cost function is evaluated
    costs = np.ones((S,), dtype=float)

    for i in range(S):
        hp = np.array(_X[i], int)  # hyperparameters are integers
        seq_sizes=hp[0]
        try:
          mae= fitness_function(int(seq_sizes))
        except:
          mae=100000

        if (mae==np.nan): mae=100000
        costs[i] = mae

    return costs


In [7]:
from pyswarms.single import GeneralOptimizerPSO
from pyswarms.backend.topology import Star, Ring, VonNeumann, Random, Pyramid
from random import random
N_PARTICLES= 7  # number of particles
# Define the bounds for seq_size

N_HYPERPARAMS= 1  # number of hyperparameters
N_SEQ=[30,70]
# Swarm parameters
# c1:cognitive parameter, c2:social parameter, w:inertia parameter, k:num neighbors, p:L1 or L2
PYSWARM_OPTS= {'c1': 0.5, 'c2': 0.5, 'w': 0.7, 'k': 3, 'p': 2}

# Constraints are alphabetical
Constraints = (np.array([N_SEQ[0], N_SEQ[1]], dtype=int))

init_pos = np.array([avg(N_SEQ)])  # middles of the contraint ranges
Init_pos= np.array([init_pos+1-2*random() for _ in range(N_PARTICLES)])

PSO_ITERS_N= 10  # number of iterations

optimizer = GeneralOptimizerPSO(  # GeneralOptimizerPSO
    n_particles= N_PARTICLES,
    dimensions= N_HYPERPARAMS,
    options= PYSWARM_OPTS,
    topology= Star(),
    bounds= Constraints,
    init_pos= Init_pos,
    )

In [ ]:
%%time
pso_log = []  # collect Pd Pf

# run the optimizer

pso_cost, pso_pos = optimizer.optimize(costf_pso, iters=PSO_ITERS_N)
# PSO best solution
print(f'cost= {pso_cost:.3f}')
pso_op_params = [int(pso_pos[_]) for _ in range(N_HYPERPARAMS)]
print(f'OP params= {pso_op_params}')

2023-12-11 21:42:13,401 - pyswarms.single.general_optimizer - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.5, 'w': 0.7, 'k': 3, 'p': 2}
pyswarms.single.general_optimizer:   0%|          |0/10

Model: "RNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, 7)]         0         
                                                                 
 conv_layer (Conv1D)         (None, None, 200)         7200      
                                                                 
 LSTM1 (LSTM)                (None, None, 200)         320800    
                                                                 
 dropout1 (Dropout)          (None, None, 200)         0         
                                                                 
 LSTM2 (LSTM)                (None, 200)               320800    
                                                                 
 dropout2 (Dropout)          (None, 200)               0         
                                                                 
 output (Dense)              (None, 30000)             60

<ipython-input-6-ecc1cea9158c>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ClosePredict2'] = fY2.cumprod().to_numpy().flatten()


Model: "RNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, 7)]         0         
                                                                 
 conv_layer (Conv1D)         (None, None, 200)         7200      
                                                                 
 LSTM1 (LSTM)                (None, None, 200)         320800    
                                                                 
 dropout1 (Dropout)          (None, None, 200)         0         
                                                                 
 LSTM2 (LSTM)                (None, 200)               320800    
                                                                 
 dropout2 (Dropout)          (None, 200)               0         
                                                                 
 output (Dense)              (None, 30000)             60

<ipython-input-6-ecc1cea9158c>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ClosePredict2'] = fY2.cumprod().to_numpy().flatten()


Model: "RNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, 7)]         0         
                                                                 
 conv_layer (Conv1D)         (None, None, 200)         7200      
                                                                 
 LSTM1 (LSTM)                (None, None, 200)         320800    
                                                                 
 dropout1 (Dropout)          (None, None, 200)         0         
                                                                 
 LSTM2 (LSTM)                (None, 200)               320800    
                                                                 
 dropout2 (Dropout)          (None, 200)               0         
                                                                 
 output (Dense)              (None, 30000)             60

In [ ]:

pso_op_params = [int(pso_pos[_]) for _ in range(N_HYPERPARAMS)]
print(f'OP params= {pso_op_params}')